In [1]:
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.image import ContainerImage
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig,Model

import sys

In [2]:
ws = Workspace.from_config()

In [3]:
webservice_name = 'churn-classifier'

run = [x for x in ws.experiments['churn-classifier'].get_runs()][0]
run_metrics = run.get_metrics()

print(run)

model = run.register_model(model_name = "churn-classifier", model_path = "outputs/model.pkl", tags={"accuracy":run_metrics["accuracy"],"python version": sys.version[0:6]})

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1,auth_enabled=True,enable_app_insights=True)

Run(Experiment: churn-classifier,
Id: 3c5d7741-4579-4f3d-ba70-4e5a4a6738d4,
Type: None,
Status: Completed)


In [4]:
myenv = Environment.get(ws,name="sklearn_22_2")
myenv.inferencing_stack_version = "latest"


inference_config = InferenceConfig(entry_script="score.py",
                                   environment=myenv)
                               
try:
    service = Webservice(ws, webservice_name)        
    service.delete()    
except Exception as e:
    print("No Existing Service")

service = Model.deploy(ws, webservice_name, [model], inference_config, deployment_config)

#service.get_logs()
service.wait_for_deployment(show_output = True)

#print(service.get_keys())
print(service.scoring_uri)

No Existing Service
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-10-15 15:52:31+00:00 Creating Container Registry if not exists.
2021-10-15 15:52:31+00:00 Registering the environment.
2021-10-15 15:52:33+00:00 Building image.